# PoC Scrapping

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)

## Desde get_games traemos:

In [2]:


# Nombre del archivo de texto que contiene el HTML
file_name = "../data/external/elemento_html_partidos_temporada_24_25.txt"

# Diccionario para almacenar los datos de los partidos
match_data = {}
try:
    # 1. Leer el contenido del archivo .txt
    with open(file_name, "r", encoding="utf-8") as f:
        html_content = f.read()

    # 2. Crear un objeto BeautifulSoup para parsear el HTML
    soup = BeautifulSoup(html_content, "html.parser")
    
    # 3. Buscar todas las filas de la tabla (<tr>)
    filas_partidos = soup.find_all("tr", role="row")
    
    print(f"Extrayendo datos de {len(filas_partidos)} partidos...")

    for fila in filas_partidos:
        # Extraer los datos de las celdas (<td>) de cada fila
        celdas = fila.find_all("td")

        # Asegurarse de que la fila tiene la estructura esperada
        if len(celdas) > 8:
            # Extraer los datos por su índice de celda
            fecha_hora = celdas[0].get_text(strip=True)[-17:]
            nombre_local = celdas[1].get_text(strip=True)
            puntos_local = celdas[3].get_text(strip=True)
            puntos_visita = celdas[4].get_text(strip=True)
            nombre_visita = celdas[6].get_text(strip=True)
            
            # El link está dentro de la celda en el índice 8 (anteriormente 9)
            link_tag = celdas[8].find("a", href=True)
            link_estadisticas = link_tag.get('href') if link_tag else None
            
            # Usar una combinación única como clave del diccionario
            match_key = f"{nombre_local} vs {nombre_visita} ({fecha_hora})"
            
            # Guardar los datos en el diccionario
            match_data[match_key] = {
                "nombre_local": nombre_local,
                "puntos_local": puntos_local,
                "nombre_visita": nombre_visita,
                "puntos_visita": puntos_visita,
                "link_estadisticas": link_estadisticas
            }
    
    print("\nDiccionario de datos de partidos creado.")
    
    # Mostrar el diccionario para su verificación
    display(match_data)
    
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{file_name}'. Asegúrate de que el archivo existe en la ruta correcta.")
except Exception as e:
    print(f"Ocurrió un error al procesar el archivo: {e}")

Extrayendo datos de 380 partidos...

Diccionario de datos de partidos creado.


{'ATENAS (C) vs BOCA (007/10/2024 22:10)': {'nombre_local': 'ATENAS (C)',
  'puntos_local': '69',
  'nombre_visita': 'BOCA',
  'puntos_visita': '81',
  'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1'},
 'OBRAS vs PLATENSE (008/10/2024 20:00)': {'nombre_local': 'OBRAS',
  'puntos_local': '94',
  'nombre_visita': 'PLATENSE',
  'puntos_visita': '90',
  'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/UYiBwuNnKVL9qCbXhOY76g==?a=1'},
 'INSTITUTO vs OLIMPICO (LB) (008/10/2024 21:00)': {'nombre_local': 'INSTITUTO',
  'puntos_local': '99',
  'nombre_visita': 'OLIMPICO (LB)',
  'puntos_visita': '73',
  'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/Dwgh5U3-8hNWcMtXCbn6yw==?a=1'},
 'UNION (SF) vs SAN LORENZO (008/10/2024 21:30)': {'nombre_local': 'UNION (SF)',
  'puntos_local': '68',
  'nombre_visita': 'SAN LORENZO',
  'puntos_visita': '63',
  'link_estadisticas': 'https://estadisticascabb.gesdepor

## Pruebas de web scrapping

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import re # Carta de trio
import json # Box score
import time # Box score


### Mapa de tiro

In [4]:
# 1. Define la clave exacta del partido que querés buscar
partido_key = 'FERRO vs PEÑAROL (MDP) (009/10/2024 20:00)'
partido_info = match_data[partido_key]
partido_info

{'nombre_local': 'FERRO',
 'puntos_local': '84',
 'nombre_visita': 'PEÑAROL (MDP)',
 'puntos_visita': '82',
 'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/7FrcQXet0O0dRypYCF4kWQ==?a=1'}

In [5]:
# Nota: Este script asume que la variable 'match_data' ya fue creada
#       y contiene los datos de los partidos, incluyendo los enlaces.
#       Por ejemplo:
# match_data = {
#     "ATENAS (C) vs BOCA (07/10/2024 22:10)": {
#         "nombre_local": "ATENAS (C)",
#         "puntos_local": "69",
#         "nombre_visita": "BOCA",
#         "puntos_visita": "81",
#         "link_estadisticas": "https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1"
#     }
#     ...
# }
# --- Parte 1: Obtener el primer enlace y nombres de equipos ---
try:
    if not match_data:
        raise ValueError("El diccionario 'match_data' está vacío o no ha sido creado. No se puede continuar.")
        
    # Obtener la clave del primer partido
    # primer_partido_key = list(match_data.keys())[0]
    # partido_info = match_data[primer_partido_key]

    partido_key = 'FERRO vs PEÑAROL (MDP) (009/10/2024 20:00)'
    partido_info = match_data[partido_key]

    # Guardar el enlace y los nombres de los equipos
    primer_link = partido_info["link_estadisticas"]
    nombre_local = partido_info["nombre_local"]
    nombre_visitante = partido_info["nombre_visita"]
    
    print(f"El enlace del primer partido es: {primer_link}")
    print(f"Procesando partido: {nombre_local} vs {nombre_visitante}")


except Exception as e:
    print(f"Ocurrió un error al obtener datos del diccionario: {e}")
    primer_link = None

# --- Parte 2: Usar Selenium para navegar a ese enlace y extraer datos ---
if primer_link:
    try:
        print("\nIniciando Selenium para navegar al mapa de tiro...")
        
        # Configuración de Selenium
        options = Options()
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        # options.add_argument("--headless")  # Descomenta si no quieres ver el navegador
        
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(primer_link)

        # Esperar el iframe principal y cambiar a él
        WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
        
        # Ahora, dentro del iframe principal, buscar el iframe del mapa de tiro
        iframe_mapa = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[src*='mapa-tiro']"))
        )
        driver.switch_to.frame(iframe_mapa)
        
        # Extraer el HTML del iframe del mapa de tiro
        html_mapa_tiro = driver.page_source
        
        # Procesar el HTML con BeautifulSoup
        soup = BeautifulSoup(html_mapa_tiro, "html.parser")
        tiros = soup.find_all("i", class_="ico-tiro")
        
        print(f"Se encontraron {len(tiros)} tiros en el mapa del primer partido.")

        # Crear un DataFrame con los datos de los tiros
        data = []
        for tiro in tiros:
            clases = tiro.get("class", [])
            estilo = tiro.get("style", "")
            
            resultado = "fallado" if "fa-times" in clases else "acertado" if "fa-circle" in clases else "desconocido"
            
            try:
                left = float(estilo.split("left:")[1].split("%")[0].strip())
                top = float(estilo.split("top:")[1].split("%")[0].strip())
            except (IndexError, ValueError):
                left, top = None, None

            # --- LÓGICA FINAL PARA DETERMINAR EL EQUIPO ---
            # Se asigna el NOMBRE del equipo local o visitante según la coordenada 'left'
            if left is not None:
                equipo = nombre_local if left <= 50 else nombre_visitante
            else:
                equipo = "desconocido"
            
            data.append({
                "equipo": equipo,
                "resultado": resultado,
                "left_pct": left,
                "top_pct": top
            })

        df_tiros = pd.DataFrame(data)
        print(f"\nDataFrame de tiros del partido ({nombre_local} vs {nombre_visitante}):")
        display(df_tiros.head(20))

    except Exception as e:
        print(f"Ocurrió un error en la parte de Selenium: {e}")
    finally:
        try:
            driver.quit()
        except NameError:
            pass

El enlace del primer partido es: https://estadisticascabb.gesdeportiva.es/partido/7FrcQXet0O0dRypYCF4kWQ==?a=1
Procesando partido: FERRO vs PEÑAROL (MDP)

Iniciando Selenium para navegar al mapa de tiro...
Se encontraron 146 tiros en el mapa del primer partido.

DataFrame de tiros del partido (FERRO vs PEÑAROL (MDP)):


,equipo,resultado,left_pct,top_pct
0,PEÑAROL (MDP),fallado,80.03,51.68
1,PEÑAROL (MDP),fallado,62.52,79.84
2,FERRO,fallado,32.87,80.36
3,PEÑAROL (MDP),fallado,70.97,14.21
4,FERRO,fallado,12.90,52.45
5,PEÑAROL (MDP),fallado,80.95,50.65
6,PEÑAROL (MDP),fallado,80.03,68.22
7,PEÑAROL (MDP),fallado,68.51,16.28
8,FERRO,fallado,12.60,53.75
9,FERRO,fallado,37.94,46.51


### Box Score

In [6]:
# --- Datos de ejemplo ---
# match_data = {
#     "ATENAS (C) vs BOCA (07/10/2024 22:10)": {
#         "link_estadisticas": "https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1"
#     }
# }
# --- Fin del ejemplo ---

all_player_stats = []
driver = None

try:
    # primer_partido_key = list(match_data.keys())[0]  # Obtener el último partido del diccionario
    # primer_link = match_data[primer_partido_key]["link_estadisticas"]

    partido_key = 'FERRO vs PEÑAROL (MDP) (009/10/2024 20:00)'
    partido_info = match_data[partido_key]["link_estadisticas"]
    print(f"El enlace del partido es: {primer_link}")

    # --- Parte 1: NAVEGACIÓN CON SELENIUM (YA FUNCIONA CORRECTAMENTE) ---
    print("\nIniciando Selenium...")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(primer_link)
    
    wait = WebDriverWait(driver, 20)

    print("Cambiando al iframe principal...")
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
    print("Cambio al iframe principal exitoso.")

    print("Buscando y haciendo clic en la pestaña 'Estadísticas'...")
    tab_selector = (By.CSS_SELECTOR, "li.pestana-estadisticas")
    estadisticas_tab_element = wait.until(EC.visibility_of_element_located(tab_selector))
    driver.execute_script("arguments[0].click();", estadisticas_tab_element)
    time.sleep(1)

    print("Cambiando al iframe de datos de estadísticas...")
    iframe_de_datos_selector = (By.CSS_SELECTOR, "div.contenido-estadisticas.activo iframe")
    wait.until(EC.frame_to_be_available_and_switch_to_it(iframe_de_datos_selector))
    print("Acceso al iframe de datos exitoso.")

    print("Extrayendo el código HTML final...")
    html_box_scores = driver.page_source
    
    # --- Parte 2: EXTRACCIÓN CON BEAUTIFULSOUP (SECCIÓN CORREGIDA) ---
    print("Procesando el HTML para extraer datos de jugadores...")
    soup = BeautifulSoup(html_box_scores, "html.parser")
    
    # --- INICIO DE LA CORRECCIÓN ---
    # Hacemos la búsqueda de los nombres de equipo de forma segura
    
    equipo_local_nombre = "Local" # Valor por defecto
    equipo_visitante_nombre = "Visitante" # Valor por defecto
    
    # Buscamos los divs que contienen los nombres de los equipos.
    # El selector 'div.nombre-equipo' es un candidato común.
    divs_nombres = soup.find_all("div", class_="nombre-equipo")
    if len(divs_nombres) >= 2:
        equipo_local_nombre = divs_nombres[0].get_text(strip=True)
        equipo_visitante_nombre = divs_nombres[1].get_text(strip=True)
    else:
        print("Advertencia: No se encontraron los nombres de equipo con el selector 'div.nombre-equipo'. Se usarán nombres genéricos.")
    # --- FIN DE LA CORRECCIÓN ---

    tablas = soup.find_all("table", class_="tabla-estadisticas")
    
    if len(tablas) >= 2:
        print(f"\nExtrayendo datos de: {equipo_local_nombre}")
        for fila in tablas[0].find("tbody").find_all("tr", onclick=True):
            onclick_attr = fila["onclick"]
            match = re.search(r"(\{.*\})", onclick_attr)
            if match:
                json_str = match.group(1).replace("'", '"')
                player_data = json.loads(json_str)
                player_data['equipo'] = equipo_local_nombre
                all_player_stats.append(player_data)

        print(f"Extrayendo datos de: {equipo_visitante_nombre}")
        for fila in tablas[1].find("tbody").find_all("tr", onclick=True):
            onclick_attr = fila["onclick"]
            match = re.search(r"(\{.*\})", onclick_attr)
            if match:
                json_str = match.group(1).replace("'", '"')
                player_data = json.loads(json_str)
                player_data['equipo'] = equipo_visitante_nombre
                all_player_stats.append(player_data)
    
    # --- Parte 3: PRESENTACIÓN DE DATOS CON PANDAS ---
    if all_player_stats:
        df_box_scores = pd.DataFrame(all_player_stats)
        print("\n✅ DataFrame final con todos los datos extraídos:")
        df_box_scores
    else:
        print("\nNo se pudieron extraer datos de jugadores del HTML.")

except Exception as e:
    print(f"Ocurrió un error general: {e}")
finally:
    if driver:
        driver.quit()
        print("\nNavegador cerrado.")

df_box_scores = df_box_scores[['IdJugador', 'IdClub', 'IdEquipo', 'Nombre', 'NombreCompleto',
                              'Puntos', 'TirosDos', 'TirosTres', 'TirosLibres',
                              'ReboteDefensivo', 'ReboteOfensivo', 'RebotesTotales',
                              'Asistencias', 'Recuperaciones', 'Perdidas',
                              'TaponCometido','TaponRecibido', 'FaltaCometida','FaltaRecibida','Valoracion',
                              'TiempoJuego', 'CincoInicial', 'equipo']]
# Lista de las columnas que contienen diccionarios para procesar
columnas_con_diccionarios = ['TirosDos', 'TirosTres', 'TirosLibres']

# Iteramos sobre cada columna que necesitamos transformar
for columna in columnas_con_diccionarios:
    if columna in df_box_scores.columns:
        # Creamos la nueva columna para 'Aciertos'
        # La función lambda toma cada diccionario (x) de la columna y extrae el valor de 'Aciertos'
        # .get('Aciertos', 0) es una forma segura que devuelve 0 si la clave 'Aciertos' no existe
        df_box_scores[f'{columna}Aciertos'] = df_box_scores[columna].apply(lambda x: x.get('Aciertos', 0))
        
        # Creamos la nueva columna para 'Fallos'
        df_box_scores[f'{columna}Fallos'] = df_box_scores[columna].apply(lambda x: x.get('Fallos', 0))

# Eliminamos las columnas originales que contenían los diccionarios para limpiar el DataFrame
df_box_scores = df_box_scores.drop(columns=columnas_con_diccionarios)
df_box_scores


El enlace del partido es: https://estadisticascabb.gesdeportiva.es/partido/7FrcQXet0O0dRypYCF4kWQ==?a=1

Iniciando Selenium...
Cambiando al iframe principal...
Cambio al iframe principal exitoso.
Buscando y haciendo clic en la pestaña 'Estadísticas'...
Cambiando al iframe de datos de estadísticas...
Acceso al iframe de datos exitoso.
Extrayendo el código HTML final...
Procesando el HTML para extraer datos de jugadores...

Extrayendo datos de: FERRO
Extrayendo datos de: PEÑAROL (MDP)

✅ DataFrame final con todos los datos extraídos:

Navegador cerrado.


,IdJugador,IdClub,IdEquipo,Nombre,NombreCompleto,Puntos,ReboteDefensivo,ReboteOfensivo,RebotesTotales,Asistencias,Recuperaciones,Perdidas,TaponCometido,TaponRecibido,FaltaCometida,FaltaRecibida,Valoracion,TiempoJuego,CincoInicial,equipo,TirosDosAciertos,TirosDosFallos,TirosTresAciertos,TirosTresFallos,TirosLibresAciertos,TirosLibresFallos
0,68608,46,0,"GONZALEZ, M.","GONZALEZ CELANO, MARTIN",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,FERRO,0,0,0,0,0,0
1,271508,46,0,"LEZCANO, E.","LEZCANO, EMILIANO",14,4,0,4,2,0,3,1,1,4,2,6,32:07,True,FERRO,7,0,0,0,0,0
2,326914,46,0,"LADO, M.","LADO, MOGGA CHARLES ARNADO",11,2,0,2,1,0,2,0,2,1,4,8,21:24,True,FERRO,4,0,0,0,3,0
3,324409,46,0,"MARTINEZ, J.","MARTINEZ, JANO DAVID",5,2,1,3,1,2,1,0,0,3,0,-1,22:18,False,FERRO,1,0,1,0,0,0
4,271509,46,0,"ROVERES, A.","ROVERES, AUGUSTO DANIEL",2,2,1,3,0,0,0,0,0,0,0,3,07:03,False,FERRO,1,0,0,0,0,0
5,74902,46,0,"RODRIGUEZ, F.","RODRIGUEZ, FELIPE SALVADOR",3,6,0,6,0,1,0,0,0,3,0,3,20:11,True,FERRO,0,0,1,0,0,0
6,182885,46,0,"WOLINSKY, F.","WOLINSKY, FACUNDO ARIEL",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,FERRO,0,0,0,0,0,0
7,278259,46,0,"GALLEGOS, R.","GALLEGOS, RODRIGO AGUSTIN",19,8,1,9,5,1,2,0,0,5,4,25,32:29,True,FERRO,6,0,0,0,7,0
8,68602,46,0,"RODRIGUEZ, C.","RODRIGUEZ, CAMILO",0,0,0,0,0,2,0,0,0,0,0,1,05:57,False,FERRO,0,0,0,0,0,0
9,325771,46,0,"DIEZ, A.","DIEZ, ALEJANDRO GABRIEL",10,3,2,5,1,0,2,0,1,3,3,10,26:35,False,FERRO,4,0,0,0,2,0


In [7]:
df_box_scores.columns

Index(['IdJugador', 'IdClub', 'IdEquipo', 'Nombre', 'NombreCompleto', 'Puntos',
       'ReboteDefensivo', 'ReboteOfensivo', 'RebotesTotales', 'Asistencias',
       'Recuperaciones', 'Perdidas', 'TaponCometido', 'TaponRecibido',
       'FaltaCometida', 'FaltaRecibida', 'Valoracion', 'TiempoJuego',
       'CincoInicial', 'equipo', 'TirosDosAciertos', 'TirosDosFallos',
       'TirosTresAciertos', 'TirosTresFallos', 'TirosLibresAciertos',
       'TirosLibresFallos'],
      dtype='object')

### Play by Play

In [8]:
acciones_del_partido = []
driver = None

try:
    # Asumiendo que 'match_data' ya fue creada
    # partido_key = list(match_data.keys())[0]
    # partido_link = match_data[partido_key]["link_estadisticas"]

    partido_key = 'FERRO vs PEÑAROL (MDP) (009/10/2024 20:00)'
    partido_link = match_data[partido_key]["link_estadisticas"]
    print(f"El enlace del partido es: {partido_link}")

    # --- Parte 1: NAVEGACIÓN CON SELENIUM ---
    print("\nIniciando Selenium...")
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(partido_link)
    
    wait = WebDriverWait(driver, 20)

    print("Cambiando al iframe principal...")
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
    print("Cambio al iframe principal exitoso.")
    
    print("Haciendo clic en la pestaña 'En vivo'...")
    en_vivo_tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li.pestana-en-vivo")))
    driver.execute_script("arguments[0].click();", en_vivo_tab)
    
    print("Cambiando al iframe anidado del Play-by-Play...")
    iframe_pbp_selector = (By.CSS_SELECTOR, "div.contenido-en-vivo div:nth-child(2) iframe")
    wait.until(EC.frame_to_be_available_and_switch_to_it(iframe_pbp_selector))
    print("Acceso al iframe final del Play-by-Play exitoso.")
    
    html_pbp = driver.page_source
    
    # --- Parte 2: EXTRACCIÓN CON BEAUTIFULSOUP (CORREGIDA) ---
    print("Procesando el HTML para extraer las acciones del partido...")
    soup = BeautifulSoup(html_pbp, "html.parser")

    contenedor_acciones = soup.find("ul", class_="listadoAccionesPartido")

    if contenedor_acciones:
        acciones = contenedor_acciones.find_all("li", class_="accion")
        print(f"Se encontraron {len(acciones)} acciones en el partido.")

        for accion in acciones:
            # Inicializamos todas las variables
            tipo_accion = jugador = cuarto = tiempo = ""
            puntos_local = None 
            puntos_visita = None

            titulo_tag = accion.find("strong", class_="titulo")
            if titulo_tag:
                tipo_accion = titulo_tag.get_text(strip=True)

            spans_info = accion.find_all("span", class_="informacion")
            if len(spans_info) >= 2:
                jugador = spans_info[0].get_text(strip=True)
                tiempo_text = spans_info[1].get_text(strip=True)
                match = re.search(r"Cuarto\s*(\d+)\s*-\s*(\d{2}:\d{2}:\d{2})", tiempo_text)
                if match:
                    cuarto = match.group(1)
                    tiempo = match.group(2)

            # --- NUEVO: Extraer el marcador de la acción ---
            marcador_tag = accion.find("strong", class_="informacionAdicional")
            if marcador_tag:
                marcador_texto = marcador_tag.get_text(strip=True)
                partes_marcador = marcador_texto.split('-')
                if len(partes_marcador) == 2:
                    try:
                        puntos_local = int(partes_marcador[0].strip())
                        puntos_visita = int(partes_marcador[1].strip())
                    except ValueError:
                        pass # Si hay un error, se quedan como None

            acciones_del_partido.append({
                "cuarto": cuarto,
                "tiempo": tiempo,
                "accion": tipo_accion,
                "jugador": jugador,
                "puntos_local": puntos_local,
                "puntos_visita": puntos_visita
            })

    # --- Parte 3: Crear y mostrar el DataFrame ---
    if acciones_del_partido:
        df_acciones = pd.DataFrame(acciones_del_partido)
        print("\n✅ DataFrame con marcador en cada acción:")
        display(df_acciones.head(10))
    else:
        print("No se pudieron extraer datos del Play-by-Play.")

except Exception as e:
    print(f"Ocurrió un error general: {e}")
finally:
    if driver:
        driver.quit()
        print("\nNavegador cerrado.")

El enlace del partido es: https://estadisticascabb.gesdeportiva.es/partido/7FrcQXet0O0dRypYCF4kWQ==?a=1

Iniciando Selenium...
Cambiando al iframe principal...
Cambio al iframe principal exitoso.
Haciendo clic en la pestaña 'En vivo'...
Cambiando al iframe anidado del Play-by-Play...
Acceso al iframe final del Play-by-Play exitoso.
Procesando el HTML para extraer las acciones del partido...
Se encontraron 606 acciones en el partido.

✅ DataFrame con marcador en cada acción:


,cuarto,tiempo,accion,jugador,puntos_local,puntos_visita
0,,,FINAL DEL PERIODO5,,NaN,NaN
1,5,00:00:01,REBOTE DEFENSIVO#9,"RODRIGUEZ, FELIPE SALVADOR",NaN,NaN
2,5,00:00:02,TIRO DE 2 FALLADO,"WALLACE, DEVANTE RASHAD-KEITH",NaN,NaN
3,5,00:00:02,REBOTE OFENSIVO#24,"WALLACE, DEVANTE RASHAD-KEITH",NaN,NaN
4,5,00:00:02,TIRO LIBRE FALLADO,"THOMAS JR, MARCUS WILEY",NaN,NaN
5,5,00:00:02,TIRO LIBRE ANOTADO,"THOMAS JR, MARCUS WILEY",84.0,82.0
6,5,00:00:02,2 TIROS LIBRES PARA EL#50,"THOMAS JR, MARCUS WILEY",NaN,NaN
7,5,00:00:02,ENTRA A PISTA EL#93,"FIERRO, MARIANO",NaN,NaN
8,5,00:00:02,ABANDONA LA PISTA EL#12,"GALLEGOS, RODRIGO AGUSTIN",NaN,NaN
9,5,00:00:02,FALTA RECIBIDA,"THOMAS JR, MARCUS WILEY",NaN,NaN



Navegador cerrado.


In [9]:
# --- Requisitos ---
# Se asume que ya existen:
# 1. df_acciones: Con las columnas básicas (cuarto, tiempo, accion, etc.).
# 2. df_box_scores: Con las columnas 'NombreCompleto' y 'equipo'.
# ------------------

print("Iniciando el proceso completo para generar quintetos por equipo...")

try:
    # === PASO 1: Crear el mapa de jugadores desde el Box Score ===
    print("Paso 1: Creando mapa de jugadores desde 'df_box_scores'...")
    if 'df_box_scores' not in locals() or df_box_scores.empty:
        raise NameError("El DataFrame 'df_box_scores' no existe o está vacío.")
    
    player_to_team_map = pd.Series(df_box_scores.equipo.values, index=df_box_scores.NombreCompleto).to_dict()
    nombre_local = df_box_scores['equipo'].unique()[0]
    print(f"Mapa de roster creado. Equipo local: {nombre_local}")

    # === PASO 2: Generar la columna temporal 'quinteto_en_cancha' ===
    print("Paso 2: Generando lista de 10 jugadores en cancha por acción...")
    # Invertimos el DataFrame para procesar en orden cronológico
    df_cronologico = df_acciones.iloc[::-1].reset_index(drop=True)

    jugadores_en_cancha = set()
    lista_de_quintetos_mixtos = []

    for index, row in df_cronologico.iterrows():
        accion = row.get('accion', '')
        jugador = row.get('jugador', '')
        accion_upper = accion.upper() if isinstance(accion, str) else ''

        # Actualizamos el set de jugadores en cancha
        if "ENTRA A PISTA" in accion_upper or "CAMBIO-ENTRA" in accion_upper:
            if pd.notna(jugador) and jugador != '':
                jugadores_en_cancha.add(jugador)
        elif "ABANDONA LA PISTA" in accion_upper or "CAMBIO-SALE" in accion_upper:
            if pd.notna(jugador) and jugador != '':
                jugadores_en_cancha.discard(jugador)
        
        lista_de_quintetos_mixtos.append(sorted(list(jugadores_en_cancha)))

    # Añadimos la columna temporal al DataFrame cronológico
    df_cronologico['quinteto_en_cancha'] = lista_de_quintetos_mixtos

    # === PASO 3: Separar el quinteto mixto en local y visitante ===
    print("Paso 3: Separando la lista de 10 jugadores en quintetos por equipo...")
    
    def dividir_quinteto(quinteto_mixto, roster_map, equipo_local_nombre):
        quinteto_local, quinteto_visitante = [], []
        for jugador in quinteto_mixto:
            if roster_map.get(jugador) == equipo_local_nombre:
                quinteto_local.append(jugador)
            else:
                quinteto_visitante.append(jugador)
        return sorted(quinteto_local), sorted(quinteto_visitante)

    # Aplicamos la función para crear las dos columnas finales
    nuevas_columnas = df_cronologico['quinteto_en_cancha'].apply(
        lambda q: pd.Series(dividir_quinteto(q, player_to_team_map, nombre_local))
    )
    nuevas_columnas.columns = ['quinteto_local', 'quinteto_visita']

    # Unimos las columnas finales al DataFrame cronológico
    df_cronologico = pd.concat([df_cronologico, nuevas_columnas], axis=1)

    # === PASO 4: Finalizar y mostrar el DataFrame ===
    # Eliminamos la columna temporal que ya no necesitamos
    df_cronologico = df_cronologico.drop(columns=['quinteto_en_cancha'])
    
    # Revertimos el orden para que coincida con el original y lo guardamos
    df_acciones_final = df_cronologico.iloc[::-1].reset_index(drop=True)

    print("\n✅ ¡Proceso completado! DataFrame final generado.")
    
    # Mostramos el resultado
    columnas_a_mostrar = [
    'cuarto',          
    'tiempo',           
    'accion',
    'jugador',
    'puntos_local',
    'puntos_visita',
    'quinteto_local',
    'quinteto_visita'
]
    display(df_acciones_final[columnas_a_mostrar].head(10))

except Exception as e:
    print(f"❌ Ocurrió un error en el script: {e}")

Iniciando el proceso completo para generar quintetos por equipo...
Paso 1: Creando mapa de jugadores desde 'df_box_scores'...
Mapa de roster creado. Equipo local: FERRO
Paso 2: Generando lista de 10 jugadores en cancha por acción...
Paso 3: Separando la lista de 10 jugadores en quintetos por equipo...

✅ ¡Proceso completado! DataFrame final generado.


,cuarto,tiempo,accion,jugador,puntos_local,puntos_visita,quinteto_local,quinteto_visita
0,,,FINAL DEL PERIODO5,,NaN,NaN,"[DIEZ, ALEJANDRO GABRIEL, FIERRO, MARIANO, LAD...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."
1,5,00:00:01,REBOTE DEFENSIVO#9,"RODRIGUEZ, FELIPE SALVADOR",NaN,NaN,"[DIEZ, ALEJANDRO GABRIEL, FIERRO, MARIANO, LAD...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."
2,5,00:00:02,TIRO DE 2 FALLADO,"WALLACE, DEVANTE RASHAD-KEITH",NaN,NaN,"[DIEZ, ALEJANDRO GABRIEL, FIERRO, MARIANO, LAD...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."
3,5,00:00:02,REBOTE OFENSIVO#24,"WALLACE, DEVANTE RASHAD-KEITH",NaN,NaN,"[DIEZ, ALEJANDRO GABRIEL, FIERRO, MARIANO, LAD...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."
4,5,00:00:02,TIRO LIBRE FALLADO,"THOMAS JR, MARCUS WILEY",NaN,NaN,"[DIEZ, ALEJANDRO GABRIEL, FIERRO, MARIANO, LAD...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."
5,5,00:00:02,TIRO LIBRE ANOTADO,"THOMAS JR, MARCUS WILEY",84.0,82.0,"[DIEZ, ALEJANDRO GABRIEL, FIERRO, MARIANO, LAD...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."
6,5,00:00:02,2 TIROS LIBRES PARA EL#50,"THOMAS JR, MARCUS WILEY",NaN,NaN,"[DIEZ, ALEJANDRO GABRIEL, FIERRO, MARIANO, LAD...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."
7,5,00:00:02,ENTRA A PISTA EL#93,"FIERRO, MARIANO",NaN,NaN,"[DIEZ, ALEJANDRO GABRIEL, FIERRO, MARIANO, LAD...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."
8,5,00:00:02,ABANDONA LA PISTA EL#12,"GALLEGOS, RODRIGO AGUSTIN",NaN,NaN,"[DIEZ, ALEJANDRO GABRIEL, LADO, MOGGA CHARLES ...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."
9,5,00:00:02,FALTA RECIBIDA,"THOMAS JR, MARCUS WILEY",NaN,NaN,"[DIEZ, ALEJANDRO GABRIEL, GALLEGOS, RODRIGO AG...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS..."


In [10]:
# --- FUNCIÓN PARA CALCULAR EL +/- ---
def calcular_plus_minus_corregido(df_enriquecido, roster_completo):
    plus_minus = {jugador: 0 for jugador in roster_completo.keys()}
    score_anterior = {'local': 0, 'visitante': 0}
    df_enriquecido[['puntos_local', 'puntos_visita']] = df_enriquecido[['puntos_local', 'puntos_visita']].ffill().fillna(0)
    
    for _, row in df_enriquecido.iterrows():
        puntos_actual_local, puntos_actual_visitante = row['puntos_local'], row['puntos_visita']
        if puntos_actual_local != score_anterior['local'] or puntos_actual_visitante != score_anterior['visitante']:
            diferencial_jugada = (puntos_actual_local - score_anterior['local']) - (puntos_actual_visitante - score_anterior['visitante'])
            for p in row['quinteto_local']:
                if p in plus_minus: plus_minus[p] += diferencial_jugada
            for p in row['quinteto_visita']:
                if p in plus_minus: plus_minus[p] -= diferencial_jugada
            score_anterior = {'local': puntos_actual_local, 'visitante': puntos_actual_visitante}
    return pd.DataFrame(list(plus_minus.items()), columns=['jugador', 'plus_minus'])

# --- NUEVA FUNCIÓN PARA CALCULAR POSESIONES ---
def calcular_posesiones(df_acciones):
    """
    Calcula las posesiones consumidas por cada jugador basándose en palabras clave.

    Args:
        df_acciones (pd.DataFrame): DataFrame con el play-by-play del partido.
            Debe contener las columnas 'accion' y 'jugador'.

    Returns:
        pd.DataFrame: Un DataFrame con las columnas 'jugador' y 'posesiones'.
    """

    # 1. Lista de acciones que cuentan como una posesión consumida
    acciones_de_posesion = [
        "TIRO DE 3 FALLADO",
        "TIRO DE 2 FALLADO",
        "2 TIROS LIBRES PARA",
        "3 TIROS LIBRES PARA",
        "TRIPLE",
        "CANASTA DE 2 PUNTOS",
        "PÉRDIDA DE BALÓN"
    ]

    # 2. Crear un patrón de texto para buscar todas las acciones a la vez
    # El '|' funciona como un 'OR' en la búsqueda de texto.
    patron_busqueda = '|'.join(acciones_de_posesion)

    # 3. Filtrar el DataFrame para obtener solo las filas donde la acción coincide
    # 'str.contains' busca el patrón en la columna 'accion'. 'na=False' evita errores con filas vacías.
    df_posesiones_consumidas = df_acciones[df_acciones['accion'].str.contains(patron_busqueda, na=False)].copy()

    # 4. Agrupar por jugador y contar cuántas de estas acciones tuvo cada uno
    conteo_posesiones = df_posesiones_consumidas.groupby('jugador').size()
    
    # 5. Convertir el resultado a un DataFrame con el formato correcto
    df_resultado = conteo_posesiones.reset_index(name='posesiones')

    return df_resultado

def contar_posesiones_jugadas_por_equipo(df_acciones, df_box_scores, player_to_team_map):
    """
    Cuenta las posesiones que ocurrieron para el equipo de un jugador mientras
    este se encontraba en la cancha.

    Args:
        df_acciones (pd.DataFrame): DataFrame con el play-by-play y quintetos.
        df_box_scores (pd.DataFrame): DataFrame con el roster de jugadores y equipos.
        player_to_team_map (dict): Diccionario que mapea 'NombreCompleto' a 'equipo'.

    Returns:
        pd.DataFrame: DataFrame con las columnas 'jugador' y 'posesiones_jugadas'.
    """

    # 1. Inicializar el contador para todos los jugadores del partido.
    roster = df_box_scores['NombreCompleto'].tolist()
    posesiones_jugadas = {jugador: 0 for jugador in roster}
    
    # Necesitamos saber cuál es el equipo local para diferenciar quintetos.
    nombre_local = df_box_scores['equipo'].unique()[0]

    # 2. Identificar las jugadas que finalizan una posesión.
    acciones_de_posesion = [
        "TIRO DE 3 FALLADO",
        "TIRO DE 2 FALLADO",
        "2 TIROS LIBRES PARA",
        "3 TIROS LIBRES PARA",
        "TRIPLE",
        "CANASTA DE 2 PUNTOS",
        "PÉRDIDA DE BALÓN"
    ]
    patron_busqueda = '|'.join(acciones_de_posesion)
    df_fines_de_posesion = df_acciones[df_acciones['accion'].str.contains(patron_busqueda, na=False)].copy()

    # 3. Iterar sobre las jugadas de fin de posesión.
    for _, jugada in df_fines_de_posesion.iterrows():
        jugador_accion = jugada.get('jugador')
        
        # Si no hay un jugador asociado a la acción, no podemos determinar el equipo.
        if not jugador_accion or pd.isna(jugador_accion):
            continue

        # 4. Determinar qué equipo tuvo la posesión.
        equipo_posesion = player_to_team_map.get(jugador_accion)
        
        # 5. Seleccionar el quinteto correcto (local o visitante).
        quinteto_del_equipo_en_posesion = []
        if equipo_posesion == nombre_local:
            quinteto_del_equipo_en_posesion = jugada['quinteto_local']
        else:
            quinteto_del_equipo_en_posesion = jugada['quinteto_visita']

        # 6. Sumar 1 al contador de cada jugador de ese quinteto.
        for jugador in quinteto_del_equipo_en_posesion:
            if jugador in posesiones_jugadas:
                posesiones_jugadas[jugador] += 1
    
    # 7. Convertir el resultado a un DataFrame.
    df_resultado = pd.DataFrame(list(posesiones_jugadas.items()), columns=['jugador', 'posesiones_jugadas'])
    return df_resultado

# --- FUNCIÓN MEJORADA PARA CALCULAR REBOTES OF/DEF DISPONIBLES ---
def calcular_rebotes_disponibles(df_acciones, df_box_scores, player_to_team_map):
    """
    Calcula la cantidad de oportunidades de rebote ofensivo y defensivo que 
    tuvo cada jugador mientras estaba en la cancha.

    Una oportunidad de rebote ocurre cuando hay un tiro de campo fallado o el
    último tiro libre de una secuencia es fallado.

    - Para el equipo que atacaba, es una oportunidad de rebote OFENSIVO.
    - Para el equipo que defendía, es una oportunidad de rebote DEFENSIVO.

    Args:
        df_acciones (pd.DataFrame): DF con el play-by-play y quintetos, ordenado cronológicamente.
        df_box_scores (pd.DataFrame): DF con el roster de jugadores y equipos.
        player_to_team_map (dict): Diccionario que mapea 'NombreCompleto' a 'equipo'.

    Returns:
        pd.DataFrame: DataFrame con las columnas 'jugador', 'reb_of_disp', y 'reb_def_disp'.
    """

    # 1. Inicializar contadores para todos los jugadores del partido.
    roster = df_box_scores['NombreCompleto'].tolist()
    # *** NUEVO: Dos diccionarios, uno para cada tipo de rebote disponible. ***
    reb_of_disponibles = {jugador: 0 for jugador in roster}
    reb_def_disponibles = {jugador: 0 for jugador in roster}
    
    nombre_local = df_box_scores['equipo'].unique()[0]
    
    # 2. Iterar sobre el índice del DataFrame para poder mirar filas futuras.
    num_acciones = len(df_acciones)
    for i in range(num_acciones):
        jugada = df_acciones.iloc[i]
        accion = jugada['accion']
        
        es_oportunidad_de_rebote = False
        
        # --- Lógica para detectar una oportunidad de rebote (sin cambios) ---
        if "TIRO DE 3 FALLADO" in accion or "TIRO DE 2 FALLADO" in accion:
            es_oportunidad_de_rebote = True
        elif "1 TIRO LIBRE PARA" in accion:
            if (i + 1 < num_acciones) and "TIRO LIBRE FALLADO" in df_acciones.iloc[i + 1]['accion']:
                es_oportunidad_de_rebote = True
        elif "2 TIROS LIBRES PARA" in accion:
            if (i + 2 < num_acciones) and "TIRO LIBRE FALLADO" in df_acciones.iloc[i + 2]['accion']:
                es_oportunidad_de_rebote = True
        elif "3 TIROS LIBRES PARA" in accion:
            if (i + 3 < num_acciones) and "TIRO LIBRE FALLADO" in df_acciones.iloc[i + 3]['accion']:
                es_oportunidad_de_rebote = True

        # 3. Si se encontró una oportunidad, se asigna a ambos equipos.
        if es_oportunidad_de_rebote:
            jugador_accion = jugada.get('jugador')
            if not jugador_accion or pd.isna(jugador_accion):
                continue

            equipo_ofensivo = player_to_team_map.get(jugador_accion)
            if not equipo_ofensivo:
                continue

            # *** NUEVO: Identificar ambos quintetos, el ofensivo y el defensivo. ***
            if equipo_ofensivo == nombre_local:
                quinteto_ofensivo = jugada['quinteto_local']
                quinteto_defensivo = jugada['quinteto_visita']
            else:
                quinteto_ofensivo = jugada['quinteto_visita']
                quinteto_defensivo = jugada['quinteto_local']

            # Sumar la oportunidad de REBOTE OFENSIVO a los jugadores del equipo atacante.
            for jugador in quinteto_ofensivo:
                if jugador in reb_of_disponibles:
                    reb_of_disponibles[jugador] += 1
            
            # Sumar la oportunidad de REBOTE DEFENSIVO a los jugadores del equipo defensor.
            for jugador in quinteto_defensivo:
                if jugador in reb_def_disponibles:
                    reb_def_disponibles[jugador] += 1

    # 4. Convertir los diccionarios en DataFrames y unirlos.
    df_ofensivos = pd.DataFrame(list(reb_of_disponibles.items()), columns=['jugador', 'rebote_of_disp'])
    df_defensivos = pd.DataFrame(list(reb_def_disponibles.items()), columns=['jugador', 'rebote_def_disp'])
    
    # Unir los dos dataframes en uno solo usando 'jugador' como clave.
    df_resultado = pd.merge(df_ofensivos, df_defensivos, on='jugador')
    
    return df_resultado

# --- FUNCIÓN PARA CALCULAR POSESIONES INDIVIDUALES ESTIMADAS ---
def calcular_posesiones_individuales(df_box_score):
    """
    Estima las posesiones finalizadas por cada jugador individualmente usando
    la fórmula de Dean Oliver.

    Args:
        df_box_score (pd.DataFrame): DataFrame que contiene las estadísticas
                                     detalladas por jugador. Debe incluir columnas
                                     de aciertos y fallos para cada tipo de tiro,
                                     'ReboteOfensivo' y 'Perdidas'.

    Returns:
        pd.DataFrame: El DataFrame original con una nueva columna llamada
                      'posesiones_estimadas'.
    """
    
    # Se crea una copia para evitar advertencias de SettingWithCopyWarning
    df = df_box_score.copy()

    # 1. Calcular Tiros de Campo Intentados (TCI)
    tci_individual = (df['TirosDosAciertos'] + df['TirosDosFallos'] +
                      df['TirosTresAciertos'] + df['TirosTresFallos'])

    # 2. Calcular Tiros Libres Intentados (TLI)
    tli_individual = df['TirosLibresAciertos'] + df['TirosLibresFallos']
    
    # 3. Aplicar la fórmula de Oliver y crear la nueva columna
    df['posesiones_estimadas'] = (
        tci_individual +
        0.44 * tli_individual -
        df['ReboteOfensivo'] +
        df['Perdidas']
    ).clip(lower=0).round(2)
    
    return df

# --- FUNCIÓN PARA CALCULAR PUNTOS EN EL ÚLTIMO CUARTO (LÓGICA MEJORADA) ---
def calcular_puntos_ultimo_cuarto(df_acciones, df_box_scores):
    """
    Calcula los puntos anotados por cada jugador en el último cuarto (Q4) y
    cualquier prórroga posterior.

    Args:
        df_acciones (pd.DataFrame): DataFrame con el play-by-play del partido.
                                    Debe contener las columnas 'accion', 'jugador' y 'cuarto'.
        df_box_scores (pd.DataFrame): DataFrame con el roster de jugadores para inicializar los contadores.

    Returns:
        pd.DataFrame: Un DataFrame con las columnas 'jugador' y 'puntos_q4_y_prorroga'.
    """

    # 1. Inicializar el contador de puntos.
    roster = df_box_scores['NombreCompleto'].tolist()
    puntos_finales = {jugador: 0 for jugador in roster}

    # 2. Filtrar las acciones que NO ocurrieron en los primeros 3 cuartos.
    cuartos_a_excluir = ['1', '2', '3']
    df_momentos_finales = df_acciones[~df_acciones['cuarto'].isin(cuartos_a_excluir)].copy()

    # 3. Definir los puntos por acción.
    # ¡Ajusta 'TIRO LIBRE ANOTADO' si en tus datos se llama de otra manera!
    puntos_por_accion = {
        "TRIPLE": 3,
        "CANASTA DE 2 PUNTOS": 2,
        "TIRO LIBRE ANOTADO": 1
    }

    # 4. Iterar sobre las jugadas de los momentos finales y sumar los puntos.
    for _, jugada in df_momentos_finales.iterrows():
        accion = jugada.get('accion')
        jugador = jugada.get('jugador')
        
        if accion in puntos_por_accion and pd.notna(jugador):
            if jugador in puntos_finales:
                puntos_finales[jugador] += puntos_por_accion[accion]
    
    # 5. Convertir a DataFrame y renombrar la columna para mayor claridad.
    df_resultado = pd.DataFrame(list(puntos_finales.items()), columns=['jugador', 'puntos_q4_y_prorroga'])

    return df_resultado

# --- FUNCIÓN PARA CALCULAR PUNTOS "CLUTCH" ---
def calcular_puntos_clutch(df_acciones, df_box_scores):
    """
    Calcula los puntos anotados por cada jugador en situaciones "clutch".

    Una situación "clutch" se define como:
    - Ocurre en los últimos 5 minutos del 4º cuarto o en cualquier prórroga.
    - La diferencia de puntos entre ambos equipos es de 5 o menos.

    Args:
        df_acciones (pd.DataFrame): DataFrame con el play-by-play. Debe contener
                                    'cuarto', 'tiempo' (formato "HH:MM:SS"),
                                    'puntos_local', 'puntos_visita', 'accion' y 'jugador'.
        df_box_scores (pd.DataFrame): DataFrame con el roster para inicializar contadores.

    Returns:
        pd.DataFrame: Un DataFrame con 'jugador' y 'puntos_clutch'.
    """

    # 1. Inicializar contadores y preparar datos.
    roster = df_box_scores['NombreCompleto'].tolist()
    puntos_clutch = {jugador: 0 for jugador in roster}
    
    df_acciones_copy = df_acciones.copy()
    df_acciones_copy[['puntos_local', 'puntos_visita']] = df_acciones_copy[['puntos_local', 'puntos_visita']].ffill().fillna(0)

    # 2. Definir los puntos por cada tipo de anotación.
    puntos_por_accion = {
        "TRIPLE": 3,
        "CANASTA DE 2 PUNTOS": 2,
        "TIRO LIBRE ANOTADO": 1
    }

    # 3. Iterar sobre cada jugada del partido para evaluar las condiciones.
    for _, jugada in df_acciones_copy.iterrows():
        # --- CONDICIÓN 1: PERÍODO DEL JUEGO (Q4 o Prórroga) ---
        if jugada['cuarto'] not in ['1', '2', '3']:
            
            # --- CONDICIÓN 2: TIEMPO RESTANTE (Últimos 5 minutos) ---
            try:
                parts = jugada['tiempo'].split(':')
                minutos = int(parts[1])
                
                if minutos < 5:
                    
                    # --- CONDICIÓN 3: MARCADOR APRETADO (Diferencia <= 5) ---
                    diferencia_puntos = abs(jugada['puntos_local'] - jugada['puntos_visita'])
                    if diferencia_puntos <= 5:
                        
                        accion = jugada.get('accion')
                        jugador = jugada.get('jugador')
                        
                        if accion in puntos_por_accion and pd.notna(jugador):
                            if jugador in puntos_clutch:
                                puntos_clutch[jugador] += puntos_por_accion[accion]
            except (ValueError, AttributeError, IndexError):
                # Ignora filas donde el formato de tiempo no es el esperado.
                continue
    
    # 4. Convertir el resultado a un DataFrame.
    df_resultado = pd.DataFrame(list(puntos_clutch.items()), columns=['jugador', 'puntos_clutch'])
    
    return df_resultado


In [11]:
# --- INICIO DEL SCRIPT DE ANÁLISIS PRINCIPAL ---
print("\nIniciando análisis avanzado de Play-by-Play...")
try:
    # --- PREPARACIÓN DE DATOS ---
    player_to_team_map = pd.Series(df_box_scores.equipo.values, index=df_box_scores.NombreCompleto).to_dict()
    df_sorted = df_acciones_final.iloc[::-1].reset_index(drop=True)
    
    # --- CÁLCULO DE MÉTRICAS AVANZADAS (DESDE FUNCIONES) ---
    df_plus_minus = calcular_plus_minus_corregido(df_sorted.copy(), player_to_team_map)
    df_posesiones_consumidas = calcular_posesiones(df_sorted.copy())
    df_posesiones_consumidas.rename(columns={'posesiones': 'posesiones_consumidas'}, inplace=True)
    df_posesiones_jugadas = contar_posesiones_jugadas_por_equipo(df_sorted.copy(), df_box_scores.copy(), player_to_team_map)
    df_rebotes_disponibles = calcular_rebotes_disponibles(df_sorted.copy(), df_box_scores.copy(), player_to_team_map)
    df_puntos_q4 = calcular_puntos_ultimo_cuarto(df_acciones_final.copy(), df_box_scores.copy())
    df_puntos_clutch = calcular_puntos_clutch(df_acciones_final.copy(), df_box_scores.copy())

    # --- UNIR ESTADÍSTICAS AVANZADAS ---
    print("\nUniendo todos los datos calculados...")
    resultado_final = df_box_scores.copy()
    
    lista_de_stats = [df_plus_minus, df_posesiones_consumidas, df_posesiones_jugadas, 
                      df_rebotes_disponibles, df_puntos_q4, df_puntos_clutch]
    
    for df_stat in lista_de_stats:
        df_stat.rename(columns={'jugador': 'NombreCompleto'}, inplace=True)
        resultado_final = pd.merge(resultado_final, df_stat, on='NombreCompleto', how='left')

    # Nota: Asegúrate que el nombre 'puntos_q4_y_prorroga' coincida con el que devuelve tu función.
    cols_a_rellenar = ['plus_minus', 'posesiones_consumidas', 'posesiones_jugadas', 
                       'reb_of_disp', 'reb_def_disp', 'puntos_q4_y_prorroga', 'puntos_clutch']
    for col in cols_a_rellenar:
        if col in resultado_final.columns:
            resultado_final[col] = resultado_final[col].fillna(0).astype(int)

    resultado_final = calcular_posesiones_individuales(resultado_final)

    # --- MOSTRAR RESULTADO FINAL ---
    print("\n✅ Resultado Final con Estadísticas Completas:")
    display(resultado_final)

except Exception as e:
    import traceback
    print(f"\n❌ Ocurrió un error en el script de análisis: {e}")
    traceback.print_exc()


Iniciando análisis avanzado de Play-by-Play...

Uniendo todos los datos calculados...

✅ Resultado Final con Estadísticas Completas:


,IdJugador,IdClub,IdEquipo,Nombre,NombreCompleto,Puntos,ReboteDefensivo,ReboteOfensivo,RebotesTotales,Asistencias,Recuperaciones,Perdidas,TaponCometido,TaponRecibido,FaltaCometida,FaltaRecibida,Valoracion,TiempoJuego,CincoInicial,equipo,TirosDosAciertos,TirosDosFallos,TirosTresAciertos,TirosTresFallos,TirosLibresAciertos,TirosLibresFallos,plus_minus,posesiones_consumidas,posesiones_jugadas,rebote_of_disp,rebote_def_disp,puntos_q4_y_prorroga,puntos_clutch,posesiones_estimadas
0,68608,46,0,"GONZALEZ, M.","GONZALEZ CELANO, MARTIN",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,FERRO,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00
1,271508,46,0,"LEZCANO, E.","LEZCANO, EMILIANO",14,4,0,4,2,0,3,1,1,4,2,6,32:07,True,FERRO,7,0,0,0,0,0,-2,18,69,31,36,6,6,10.00
2,326914,46,0,"LADO, M.","LADO, MOGGA CHARLES ARNADO",11,2,0,2,1,0,2,0,2,1,4,8,21:24,True,FERRO,4,0,0,0,3,0,2,12,49,21,25,1,1,7.32
3,324409,46,0,"MARTINEZ, J.","MARTINEZ, JANO DAVID",5,2,1,3,1,2,1,0,0,3,0,-1,22:18,False,FERRO,1,0,1,0,0,0,-6,11,48,24,23,0,0,2.00
4,271509,46,0,"ROVERES, A.","ROVERES, AUGUSTO DANIEL",2,2,1,3,0,0,0,0,0,0,0,3,07:03,False,FERRO,1,0,0,0,0,0,-4,3,16,10,8,0,0,0.00
5,74902,46,0,"RODRIGUEZ, F.","RODRIGUEZ, FELIPE SALVADOR",3,6,0,6,0,1,0,0,0,3,0,3,20:11,True,FERRO,0,0,1,0,0,0,8,5,44,19,24,0,0,1.00
6,182885,46,0,"WOLINSKY, F.","WOLINSKY, FACUNDO ARIEL",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,FERRO,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00
7,278259,46,0,"GALLEGOS, R.","GALLEGOS, RODRIGO AGUSTIN",19,8,1,9,5,1,2,0,0,5,4,25,32:29,True,FERRO,6,0,0,0,7,0,5,17,64,28,34,9,9,10.08
8,68602,46,0,"RODRIGUEZ, C.","RODRIGUEZ, CAMILO",0,0,0,0,0,2,0,0,0,0,0,1,05:57,False,FERRO,0,0,0,0,0,0,1,1,16,9,7,0,0,0.00
9,325771,46,0,"DIEZ, A.","DIEZ, ALEJANDRO GABRIEL",10,3,2,5,1,0,2,0,1,3,3,10,26:35,False,FERRO,4,0,0,0,2,0,6,10,64,29,39,4,2,4.88


TODO: pensar
- con estas variables puedo calcular los four factors? si
- cuantos registros tendra mi base? 380 partidos x24 jugadores = 9.120 es poco
- commo performara el group by? es necesario spark? creeria que no hace falta spark

## Iteración

In [12]:
from tqdm import tqdm
def extraer_boxscore_y_pbp(partido_link):
    """Extrae boxscore y play-by-play de un partido dado su link."""
    all_player_stats = []
    acciones_del_partido = []
    driver = None
    df_box_scores = None
    df_acciones_final = None

    try:
        # --- BOX SCORE ---
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(partido_link)
        wait = WebDriverWait(driver, 20)
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
        tab_selector = (By.CSS_SELECTOR, "li.pestana-estadisticas")
        estadisticas_tab_element = wait.until(EC.visibility_of_element_located(tab_selector))
        driver.execute_script("arguments[0].click();", estadisticas_tab_element)
        time.sleep(1)
        iframe_de_datos_selector = (By.CSS_SELECTOR, "div.contenido-estadisticas.activo iframe")
        wait.until(EC.frame_to_be_available_and_switch_to_it(iframe_de_datos_selector))
        html_box_scores = driver.page_source
        soup = BeautifulSoup(html_box_scores, "html.parser")
        divs_nombres = soup.find_all("div", class_="nombre-equipo")
        equipo_local_nombre = divs_nombres[0].get_text(strip=True) if len(divs_nombres) >= 2 else "Local"
        equipo_visitante_nombre = divs_nombres[1].get_text(strip=True) if len(divs_nombres) >= 2 else "Visitante"
        tablas = soup.find_all("table", class_="tabla-estadisticas")
        if len(tablas) >= 2:
            for idx, equipo_nombre in enumerate([equipo_local_nombre, equipo_visitante_nombre]):
                for fila in tablas[idx].find("tbody").find_all("tr", onclick=True):
                    onclick_attr = fila["onclick"]
                    match = re.search(r"(\{.*\})", onclick_attr)
                    if match:
                        json_str = match.group(1).replace("'", '"')
                        player_data = json.loads(json_str)
                        player_data['equipo'] = equipo_nombre
                        all_player_stats.append(player_data)
        if all_player_stats:
            df_box_scores = pd.DataFrame(all_player_stats)
            columnas = ['IdJugador', 'IdClub', 'IdEquipo', 'Nombre', 'NombreCompleto',
                        'Puntos', 'TirosDos', 'TirosTres', 'TirosLibres',
                        'ReboteDefensivo', 'ReboteOfensivo', 'RebotesTotales',
                        'Asistencias', 'Recuperaciones', 'Perdidas',
                        'TaponCometido','TaponRecibido', 'FaltaCometida','FaltaRecibida','Valoracion',
                        'TiempoJuego', 'CincoInicial', 'equipo']
            df_box_scores = df_box_scores[[col for col in columnas if col in df_box_scores.columns]]
            columnas_con_diccionarios = ['TirosDos', 'TirosTres', 'TirosLibres']
            for columna in columnas_con_diccionarios:
                if columna in df_box_scores.columns:
                    df_box_scores[f'{columna}Aciertos'] = df_box_scores[columna].apply(lambda x: x.get('Aciertos', 0))
                    df_box_scores[f'{columna}Fallos'] = df_box_scores[columna].apply(lambda x: x.get('Fallos', 0))
            df_box_scores = df_box_scores.drop(columns=[c for c in columnas_con_diccionarios if c in df_box_scores.columns])
        else:
            return None, None

        # --- PLAY BY PLAY ---
        driver.switch_to.default_content()
        driver.get(partido_link)
        wait = WebDriverWait(driver, 20)
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
        en_vivo_tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li.pestana-en-vivo")))
        driver.execute_script("arguments[0].click();", en_vivo_tab)
        iframe_pbp_selector = (By.CSS_SELECTOR, "div.contenido-en-vivo div:nth-child(2) iframe")
        wait.until(EC.frame_to_be_available_and_switch_to_it(iframe_pbp_selector))
        html_pbp = driver.page_source
        soup = BeautifulSoup(html_pbp, "html.parser")
        contenedor_acciones = soup.find("ul", class_="listadoAccionesPartido")
        if contenedor_acciones:
            acciones = contenedor_acciones.find_all("li", class_="accion")
            for accion in acciones:
                tipo_accion = jugador = cuarto = tiempo = ""
                puntos_local = None 
                puntos_visita = None
                titulo_tag = accion.find("strong", class_="titulo")
                if titulo_tag:
                    tipo_accion = titulo_tag.get_text(strip=True)
                spans_info = accion.find_all("span", class_="informacion")
                if len(spans_info) >= 2:
                    jugador = spans_info[0].get_text(strip=True)
                    tiempo_text = spans_info[1].get_text(strip=True)
                    match = re.search(r"Cuarto\s*(\d+)\s*-\s*(\d{2}:\d{2}:\d{2})", tiempo_text)
                    if match:
                        cuarto = match.group(1)
                        tiempo = match.group(2)
                marcador_tag = accion.find("strong", class_="informacionAdicional")
                if marcador_tag:
                    marcador_texto = marcador_tag.get_text(strip=True)
                    partes_marcador = marcador_texto.split('-')
                    if len(partes_marcador) == 2:
                        try:
                            puntos_local = int(partes_marcador[0].strip())
                            puntos_visita = int(partes_marcador[1].strip())
                        except ValueError:
                            pass
                acciones_del_partido.append({
                    "cuarto": cuarto,
                    "tiempo": tiempo,
                    "accion": tipo_accion,
                    "jugador": jugador,
                    "puntos_local": puntos_local,
                    "puntos_visita": puntos_visita
                })
        if acciones_del_partido:
            df_acciones = pd.DataFrame(acciones_del_partido)
        else:
            return df_box_scores, None

        # --- GENERAR QUINTETOS ---
        player_to_team_map = pd.Series(df_box_scores.equipo.values, index=df_box_scores.NombreCompleto).to_dict()
        nombre_local = df_box_scores['equipo'].unique()[0]
        df_cronologico = df_acciones.iloc[::-1].reset_index(drop=True)
        jugadores_en_cancha = set()
        lista_de_quintetos_mixtos = []
        for _, row in df_cronologico.iterrows():
            accion = row.get('accion', '')
            jugador = row.get('jugador', '')
            accion_upper = accion.upper() if isinstance(accion, str) else ''
            if "ENTRA A PISTA" in accion_upper or "CAMBIO-ENTRA" in accion_upper:
                if pd.notna(jugador) and jugador != '':
                    jugadores_en_cancha.add(jugador)
            elif "ABANDONA LA PISTA" in accion_upper or "CAMBIO-SALE" in accion_upper:
                if pd.notna(jugador) and jugador != '':
                    jugadores_en_cancha.discard(jugador)
            lista_de_quintetos_mixtos.append(sorted(list(jugadores_en_cancha)))
        df_cronologico['quinteto_en_cancha'] = lista_de_quintetos_mixtos
        def dividir_quinteto(quinteto_mixto, roster_map, equipo_local_nombre):
            quinteto_local, quinteto_visita = [], []
            for jugador in quinteto_mixto:
                if roster_map.get(jugador) == equipo_local_nombre:
                    quinteto_local.append(jugador)
                else:
                    quinteto_visita.append(jugador)
            return sorted(quinteto_local), sorted(quinteto_visita)
        nuevas_columnas = df_cronologico['quinteto_en_cancha'].apply(
            lambda q: pd.Series(dividir_quinteto(q, player_to_team_map, nombre_local))
        )
        nuevas_columnas.columns = ['quinteto_local', 'quinteto_visita']
        df_cronologico = pd.concat([df_cronologico, nuevas_columnas], axis=1)
        df_cronologico = df_cronologico.drop(columns=['quinteto_en_cancha'])
        df_acciones_final = df_cronologico.iloc[::-1].reset_index(drop=True)
        return df_box_scores, df_acciones_final
    except Exception as e:
        print(f"Error procesando partido: {e}")
        return None, None
    finally:
        if driver:
            driver.quit()

# --- PROCESAR LOS PRIMEROS 10 PARTIDOS ---
resultados = []
llaves_partidos = list(match_data.keys())[:10]
for partido_key in tqdm(llaves_partidos, desc="Procesando partidos"):
    partido_link = match_data[partido_key]["link_estadisticas"]
    df_box_scores, df_acciones_final = extraer_boxscore_y_pbp(partido_link)
    if df_box_scores is None or df_acciones_final is None:
        continue
    player_to_team_map = pd.Series(df_box_scores.equipo.values, index=df_box_scores.NombreCompleto).to_dict()
    df_sorted = df_acciones_final.iloc[::-1].reset_index(drop=True)
    df_plus_minus = calcular_plus_minus_corregido(df_sorted.copy(), player_to_team_map)
    df_posesiones_consumidas = calcular_posesiones(df_sorted.copy())
    df_posesiones_consumidas.rename(columns={'posesiones': 'posesiones_consumidas'}, inplace=True)
    df_posesiones_jugadas = contar_posesiones_jugadas_por_equipo(df_sorted.copy(), df_box_scores.copy(), player_to_team_map)
    df_rebotes_disponibles = calcular_rebotes_disponibles(df_sorted.copy(), df_box_scores.copy(), player_to_team_map)
    df_puntos_q4 = calcular_puntos_ultimo_cuarto(df_acciones_final.copy(), df_box_scores.copy())
    df_puntos_clutch = calcular_puntos_clutch(df_acciones_final.copy(), df_box_scores.copy())
    resultado_final = df_box_scores.copy()
    lista_de_stats = [df_plus_minus, df_posesiones_consumidas, df_posesiones_jugadas, 
                        df_rebotes_disponibles, df_puntos_q4, df_puntos_clutch]
    for df_stat in lista_de_stats:
        df_stat.rename(columns={'jugador': 'NombreCompleto'}, inplace=True)
        resultado_final = pd.merge(resultado_final, df_stat, on='NombreCompleto', how='left')
    cols_a_rellenar = ['plus_minus', 'posesiones_consumidas', 'posesiones_jugadas', 
                        'rebote_of_disp', 'rebote_def_disp', 'puntos_q4_y_prorroga', 'puntos_clutch']
    for col in cols_a_rellenar:
        if col in resultado_final.columns:
            resultado_final[col] = resultado_final[col].fillna(0).astype(int)
    resultado_final = calcular_posesiones_individuales(resultado_final)
    resultado_final['partido_key'] = partido_key
    resultados.append(resultado_final)

# --- CONCATENAR TODOS LOS RESULTADOS ---
if resultados:
    df_resultado_final = pd.concat(resultados, ignore_index=True)
    print("\n✅ DataFrame final con todos los registros de todos los jugadores de los primeros 10 partidos.")
    display(df_resultado_final)
else:
    print("No se pudieron procesar partidos correctamente.")

Procesando partidos: 100%|██████████| 10/10 [04:27<00:00, 26.73s/it]


✅ DataFrame final con todos los registros de todos los jugadores de los primeros 10 partidos.


,IdJugador,IdClub,IdEquipo,Nombre,NombreCompleto,Puntos,ReboteDefensivo,ReboteOfensivo,RebotesTotales,Asistencias,Recuperaciones,Perdidas,TaponCometido,TaponRecibido,FaltaCometida,FaltaRecibida,Valoracion,TiempoJuego,CincoInicial,equipo,TirosDosAciertos,TirosDosFallos,TirosTresAciertos,TirosTresFallos,TirosLibresAciertos,TirosLibresFallos,plus_minus,posesiones_consumidas,posesiones_jugadas,rebote_of_disp,rebote_def_disp,puntos_q4_y_prorroga,puntos_clutch,posesiones_estimadas,partido_key
0,78377,1498,0,"ARAUJO, M.","ARAUJO, MAXIMO",3,0,0,0,0,0,0,0,0,1,0,1,18:05,False,ATENAS (C),0,0,1,0,0,0,1,2,53,26,22,0,0,1.00,ATENAS (C) vs BOCA (007/10/2024 22:10)
1,326699,1498,0,"BUENDIA, C.","BUENDIA, CARLOS MANUEL",1,0,0,0,1,0,1,0,0,0,3,1,07:57,True,ATENAS (C),0,0,0,0,1,0,-15,4,14,7,4,0,0,1.44,ATENAS (C) vs BOCA (007/10/2024 22:10)
2,273565,1498,0,"MONTERO, J.","MONTERO, JOSE IGNACIO",2,0,0,0,3,1,2,0,0,0,2,4,23:27,False,ATENAS (C),0,0,0,0,2,0,-2,5,44,19,17,0,0,2.88,ATENAS (C) vs BOCA (007/10/2024 22:10)
3,209515,1498,0,"ARN BUSTAMANTE, L.","ARN BUSTAMANTE, LUCAS MARTIN",12,1,0,1,4,0,0,0,0,2,3,9,30:14,True,ATENAS (C),1,0,3,0,1,0,-15,13,56,27,21,0,0,4.44,ATENAS (C) vs BOCA (007/10/2024 22:10)
4,321117,1498,0,"MARCONETTI, S.","MARCONETTI, SANTIAGO JAVIER",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,ATENAS (C),0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,ATENAS (C) vs BOCA (007/10/2024 22:10)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,326165,2314,0,"SAFAR, S.","SAFAR, SELEM",8,3,0,3,1,2,3,0,0,0,1,3,26:56,True,ZARATE BASKET,1,0,2,0,0,0,-21,15,58,31,19,0,0,6.00,BOCA vs ZARATE BASKET (012/10/2024 11:30)
235,274496,2314,0,"SCIAMANNA, G.","SCIAMANNA, GINO",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,ZARATE BASKET,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,BOCA vs ZARATE BASKET (012/10/2024 11:30)
236,326162,2314,0,"FRAGA, L.","FRAGA PEREZ, LAUTARO EZEQUIEL",8,0,1,1,2,0,1,0,0,2,0,4,16:38,False,ZARATE BASKET,1,0,2,0,0,0,-7,8,34,19,15,0,0,3.00,BOCA vs ZARATE BASKET (012/10/2024 11:30)
237,326446,2314,0,"DEL DO , F.","DEL DO, FACUNDO",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,ZARATE BASKET,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,BOCA vs ZARATE BASKET (012/10/2024 11:30)


NOTA: hasta aca funciona, pensar como hacer para todos los partidos

## Iteracion 2

In [14]:
from tqdm import tqdm
import os

def extraer_boxscore_y_pbp(partido_link):
    """Extrae boxscore y play-by-play de un partido dado su link."""
    all_player_stats = []
    acciones_del_partido = []
    driver = None
    df_box_scores = None
    df_acciones_final = None

    try:
        # --- BOX SCORE ---
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(partido_link)
        wait = WebDriverWait(driver, 20)
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
        tab_selector = (By.CSS_SELECTOR, "li.pestana-estadisticas")
        estadisticas_tab_element = wait.until(EC.visibility_of_element_located(tab_selector))
        driver.execute_script("arguments[0].click();", estadisticas_tab_element)
        time.sleep(1)
        iframe_de_datos_selector = (By.CSS_SELECTOR, "div.contenido-estadisticas.activo iframe")
        wait.until(EC.frame_to_be_available_and_switch_to_it(iframe_de_datos_selector))
        html_box_scores = driver.page_source
        soup = BeautifulSoup(html_box_scores, "html.parser")
        divs_nombres = soup.find_all("div", class_="nombre-equipo")
        equipo_local_nombre = divs_nombres[0].get_text(strip=True) if len(divs_nombres) >= 2 else "Local"
        equipo_visitante_nombre = divs_nombres[1].get_text(strip=True) if len(divs_nombres) >= 2 else "Visitante"
        tablas = soup.find_all("table", class_="tabla-estadisticas")
        if len(tablas) >= 2:
            for idx, equipo_nombre in enumerate([equipo_local_nombre, equipo_visitante_nombre]):
                for fila in tablas[idx].find("tbody").find_all("tr", onclick=True):
                    onclick_attr = fila["onclick"]
                    match = re.search(r"(\{.*\})", onclick_attr)
                    if match:
                        json_str = match.group(1).replace("'", '"')
                        player_data = json.loads(json_str)
                        player_data['equipo'] = equipo_nombre
                        all_player_stats.append(player_data)
        if all_player_stats:
            df_box_scores = pd.DataFrame(all_player_stats)
            columnas = ['IdJugador', 'IdClub', 'IdEquipo', 'Nombre', 'NombreCompleto',
                        'Puntos', 'TirosDos', 'TirosTres', 'TirosLibres',
                        'ReboteDefensivo', 'ReboteOfensivo', 'RebotesTotales',
                        'Asistencias', 'Recuperaciones', 'Perdidas',
                        'TaponCometido','TaponRecibido', 'FaltaCometida','FaltaRecibida','Valoracion',
                        'TiempoJuego', 'CincoInicial', 'equipo']
            df_box_scores = df_box_scores[[col for col in columnas if col in df_box_scores.columns]]
            columnas_con_diccionarios = ['TirosDos', 'TirosTres', 'TirosLibres']
            for columna in columnas_con_diccionarios:
                if columna in df_box_scores.columns:
                    df_box_scores[f'{columna}Aciertos'] = df_box_scores[columna].apply(lambda x: x.get('Aciertos', 0))
                    df_box_scores[f'{columna}Fallos'] = df_box_scores[columna].apply(lambda x: x.get('Fallos', 0))
            df_box_scores = df_box_scores.drop(columns=[c for c in columnas_con_diccionarios if c in df_box_scores.columns])
        else:
            return None, None

        # --- PLAY BY PLAY ---
        driver.switch_to.default_content()
        driver.get(partido_link)
        wait = WebDriverWait(driver, 20)
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
        en_vivo_tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li.pestana-en-vivo")))
        driver.execute_script("arguments[0].click();", en_vivo_tab)
        iframe_pbp_selector = (By.CSS_SELECTOR, "div.contenido-en-vivo div:nth-child(2) iframe")
        wait.until(EC.frame_to_be_available_and_switch_to_it(iframe_pbp_selector))
        html_pbp = driver.page_source
        soup = BeautifulSoup(html_pbp, "html.parser")
        contenedor_acciones = soup.find("ul", class_="listadoAccionesPartido")
        if contenedor_acciones:
            acciones = contenedor_acciones.find_all("li", class_="accion")
            for accion in acciones:
                tipo_accion = jugador = cuarto = tiempo = ""
                puntos_local = None 
                puntos_visita = None
                titulo_tag = accion.find("strong", class_="titulo")
                if titulo_tag:
                    tipo_accion = titulo_tag.get_text(strip=True)
                spans_info = accion.find_all("span", class_="informacion")
                if len(spans_info) >= 2:
                    jugador = spans_info[0].get_text(strip=True)
                    tiempo_text = spans_info[1].get_text(strip=True)
                    match = re.search(r"Cuarto\s*(\d+)\s*-\s*(\d{2}:\d{2}:\d{2})", tiempo_text)
                    if match:
                        cuarto = match.group(1)
                        tiempo = match.group(2)
                marcador_tag = accion.find("strong", class_="informacionAdicional")
                if marcador_tag:
                    marcador_texto = marcador_tag.get_text(strip=True)
                    partes_marcador = marcador_texto.split('-')
                    if len(partes_marcador) == 2:
                        try:
                            puntos_local = int(partes_marcador[0].strip())
                            puntos_visita = int(partes_marcador[1].strip())
                        except ValueError:
                            pass
                acciones_del_partido.append({
                    "cuarto": cuarto,
                    "tiempo": tiempo,
                    "accion": tipo_accion,
                    "jugador": jugador,
                    "puntos_local": puntos_local,
                    "puntos_visita": puntos_visita
                })
        if acciones_del_partido:
            df_acciones = pd.DataFrame(acciones_del_partido)
        else:
            return df_box_scores, None

        # --- GENERAR QUINTETOS ---
        player_to_team_map = pd.Series(df_box_scores.equipo.values, index=df_box_scores.NombreCompleto).to_dict()
        nombre_local = df_box_scores['equipo'].unique()[0]
        df_cronologico = df_acciones.iloc[::-1].reset_index(drop=True)
        jugadores_en_cancha = set()
        lista_de_quintetos_mixtos = []
        for _, row in df_cronologico.iterrows():
            accion = row.get('accion', '')
            jugador = row.get('jugador', '')
            accion_upper = accion.upper() if isinstance(accion, str) else ''
            if "ENTRA A PISTA" in accion_upper or "CAMBIO-ENTRA" in accion_upper:
                if pd.notna(jugador) and jugador != '':
                    jugadores_en_cancha.add(jugador)
            elif "ABANDONA LA PISTA" in accion_upper or "CAMBIO-SALE" in accion_upper:
                if pd.notna(jugador) and jugador != '':
                    jugadores_en_cancha.discard(jugador)
            lista_de_quintetos_mixtos.append(sorted(list(jugadores_en_cancha)))
        df_cronologico['quinteto_en_cancha'] = lista_de_quintetos_mixtos
        def dividir_quinteto(quinteto_mixto, roster_map, equipo_local_nombre):
            quinteto_local, quinteto_visita = [], []
            for jugador in quinteto_mixto:
                if roster_map.get(jugador) == equipo_local_nombre:
                    quinteto_local.append(jugador)
                else:
                    quinteto_visita.append(jugador)
            return sorted(quinteto_local), sorted(quinteto_visita)
        nuevas_columnas = df_cronologico['quinteto_en_cancha'].apply(
            lambda q: pd.Series(dividir_quinteto(q, player_to_team_map, nombre_local))
        )
        nuevas_columnas.columns = ['quinteto_local', 'quinteto_visita']
        df_cronologico = pd.concat([df_cronologico, nuevas_columnas], axis=1)
        df_cronologico = df_cronologico.drop(columns=['quinteto_en_cancha'])
        df_acciones_final = df_cronologico.iloc[::-1].reset_index(drop=True)
        return df_box_scores, df_acciones_final
    except Exception as e:
        print(f"Error procesando partido: {e}")
        return None, None
    finally:
        if driver:
            driver.quit()

# --- PROCESAR LOS PARTIDOS Y GUARDAR INDIVIDUALMENTE ---
# Asegúrate de que `match_data` esté definido y sea un diccionario con las llaves de los partidos y los links.
# Por ejemplo: match_data = {'partido_1': {'link_estadisticas': 'http://link1.com'}, ...}

# Crear la carpeta si no existe
output_dir = "data/scrapped"
os.makedirs(output_dir, exist_ok=True)
llaves_partidos = list(match_data.keys())
partidos_procesados = 0

def sanitize_filename(filename):
    """
    Limpia una cadena de texto para que sea un nombre de archivo válido.
    Reemplaza caracteres no permitidos como :, /, \, etc., con un guion bajo.
    """
    # Expresión regular que coincide con caracteres no válidos para nombres de archivo en Windows
    # y la mayoría de los sistemas.
    invalid_chars = re.compile(r'[\\/:*?"<>|]')
    return invalid_chars.sub('_', filename)

for partido_key in tqdm(llaves_partidos, desc="Procesando partidos"):
    # Comprobar si ya se procesó el partido (opcional, para reanudar)
    # Por ejemplo, si tienes un archivo de registro
    # if partido_key in registro_partidos_completados:
    #   continue

    partido_link = match_data[partido_key]["link_estadisticas"]
    df_box_scores, df_acciones_final = extraer_boxscore_y_pbp(partido_link)
    
    if df_box_scores is None or df_acciones_final is None:
        print(f"Partido {partido_key} fallido. Continuando...")
        partidos_procesados += 1
        continue
    
    # Procesar los datos si la extracción fue exitosa
    player_to_team_map = pd.Series(df_box_scores.equipo.values, index=df_box_scores.NombreCompleto).to_dict()
    df_sorted = df_acciones_final.iloc[::-1].reset_index(drop=True)
    
    try:
        df_plus_minus = calcular_plus_minus_corregido(df_sorted.copy(), player_to_team_map)
        df_posesiones_consumidas = calcular_posesiones(df_sorted.copy())
        df_posesiones_consumidas.rename(columns={'posesiones': 'posesiones_consumidas'}, inplace=True)
        df_posesiones_jugadas = contar_posesiones_jugadas_por_equipo(df_sorted.copy(), df_box_scores.copy(), player_to_team_map)
        df_rebotes_disponibles = calcular_rebotes_disponibles(df_sorted.copy(), df_box_scores.copy(), player_to_team_map)
        df_puntos_q4 = calcular_puntos_ultimo_cuarto(df_acciones_final.copy(), df_box_scores.copy())
        df_puntos_clutch = calcular_puntos_clutch(df_acciones_final.copy(), df_box_scores.copy())
    except Exception as e:
        print(f"Error en los cálculos para el partido {partido_key}: {e}")
        partidos_procesados += 1
        continue

    resultado_final = df_box_scores.copy()
    lista_de_stats = [df_plus_minus, df_posesiones_consumidas, df_posesiones_jugadas, 
                        df_rebotes_disponibles, df_puntos_q4, df_puntos_clutch]
    
    for df_stat in lista_de_stats:
        if not df_stat.empty and 'jugador' in df_stat.columns:
            df_stat.rename(columns={'jugador': 'NombreCompleto'}, inplace=True)
            resultado_final = pd.merge(resultado_final, df_stat, on='NombreCompleto', how='left')

    cols_a_rellenar = ['plus_minus', 'posesiones_consumidas', 'posesiones_jugadas', 
                        'rebote_of_disp', 'rebote_def_disp', 'puntos_q4_y_prorroga', 'puntos_clutch']
    
    for col in cols_a_rellenar:
        if col in resultado_final.columns:
            resultado_final[col] = resultado_final[col].fillna(0).astype(int)
    
    resultado_final = calcular_posesiones_individuales(resultado_final)
    resultado_final['partido_key'] = partido_key

    # Limpiar la llave del partido antes de usarla en el nombre del archivo
    clean_partido_key = sanitize_filename(partido_key)
    
    # Guardar cada DataFrame en un archivo parquet individual
    nombre_archivo = f"partido_{partidos_procesados + 1}_{clean_partido_key}.parquet"
    ruta_archivo = os.path.join(output_dir, nombre_archivo)
    resultado_final.to_parquet(ruta_archivo, index=False)
    
    print(f"\n✅ Partido {partidos_procesados + 1} ({partido_key}) procesado y guardado en: {ruta_archivo}")
    partidos_procesados += 1
    
if partidos_procesados > 0:
    print(f"\n✅ Proceso completado. Se procesaron {partidos_procesados} partidos y se guardaron en la carpeta '{output_dir}'.")
else:
    print("No se pudieron procesar partidos correctamente.")

<>:169: SyntaxWarning: invalid escape sequence '\,'
<>:169: SyntaxWarning: invalid escape sequence '\,'
C:\Users\roman.landa\AppData\Local\Temp\ipykernel_34528\2630994244.py:169: SyntaxWarning: invalid escape sequence '\,'
  Reemplaza caracteres no permitidos como :, /, \, etc., con un guion bajo.
Procesando partidos:   0%|          | 1/380 [01:26<9:04:27, 86.19s/it]


✅ Partido 1 (ATENAS (C) vs BOCA (007/10/2024 22:10)) procesado y guardado en: data/scrapped\partido_1_ATENAS (C) vs BOCA (007_10_2024 22_10).parquet


Procesando partidos:   1%|          | 2/380 [02:02<5:57:14, 56.70s/it]


✅ Partido 2 (OBRAS vs PLATENSE (008/10/2024 20:00)) procesado y guardado en: data/scrapped\partido_2_OBRAS vs PLATENSE (008_10_2024 20_00).parquet


Procesando partidos:   1%|          | 3/380 [02:31<4:37:45, 44.20s/it]

...


✅ Partido 378 (SAN LORENZO vs FERRO (020/05/2025 20:00)) procesado y guardado en: data/scrapped\partido_378_SAN LORENZO vs FERRO (020_05_2025 20_00).parquet


Procesando partidos: 100%|█████████▉| 379/380 [8:20:16<00:32, 32.42s/it]


✅ Partido 379 (BOCA vs ARGENTINO (J) (020/05/2025 20:30)) procesado y guardado en: data/scrapped\partido_379_BOCA vs ARGENTINO (J) (020_05_2025 20_30).parquet


Procesando partidos: 100%|██████████| 380/380 [8:20:41<00:00, 79.06s/it]


✅ Partido 380 (OBERA vs LA UNION FSA. (020/05/2025 21:00)) procesado y guardado en: data/scrapped\partido_380_OBERA vs LA UNION FSA. (020_05_2025 21_00).parquet

✅ Proceso completado. Se procesaron 380 partidos y se guardaron en la carpeta 'data/scrapped'.
